In [79]:
import joblib
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tqdm import tqdm  # Para barra de progresso

# 1. Carregar dados originais
print("Carregando dados...")
try:
    df_original = joblib.load('df_produtos.z')
    print(f"✔ Dados carregados. Total de linhas: {len(df_original):,}")
except Exception as e:
    print(f"Erro ao carregar arquivo: {e}")
    raise

# 2. Pré-processamento
print("\nProcessando dados...")
df_processado = df_original.copy()

# Converter colunas textuais para minúsculas
text_cols = ['descricao', 'unidade', 'marca', 'categoria']
for col in tqdm(text_cols, desc="Convertendo texto"):
    if col in df_processado.columns:
        df_processado[col] = df_processado[col].str.lower()

# Processar colunas numéricas
if 'preco_venda' in df_processado.columns:
    # Padronização
    scaler_pad = StandardScaler()
    df_processado['preco_padronizado'] = scaler_pad.fit_transform(df_processado[['preco_venda']])

    # Normalização
    scaler_norm = MinMaxScaler()
    df_processado['preco_normalizado'] = scaler_norm.fit_transform(df_processado[['preco_venda']])

# 3. Verificação de qualidade
print("\nVerificando transformações...")

# Verificar minúsculas
text_check = df_processado[text_cols].apply(lambda x: x.str.islower().all())
print("\nVerificação de minúsculas:")
print(text_check)

# Verificar transformações numéricas
if 'preco_venda' in df_processado.columns:
    print("\nEstatísticas dos preços:")
    stats = df_processado[['preco_venda', 'preco_padronizado', 'preco_normalizado']].describe().round(2)
    print(stats)

    # Verificar se a padronização funcionou (média ~0, std ~1)
    pad_mean = df_processado['preco_padronizado'].mean()
    pad_std = df_processado['preco_padronizado'].std()
    print(f"\nPadronização - Média: {pad_mean:.2f} (deveria ser ~0), Desvio: {pad_std:.2f} (deveria ser ~1)")

    # Verificar normalização (valores entre 0-1)
    norm_min = df_processado['preco_normalizado'].min()
    norm_max = df_processado['preco_normalizado'].max()
    print(f"Normalização - Mínimo: {norm_min:.2f}, Máximo: {norm_max:.2f} (deveria estar entre 0-1)")

# 4. Salvar resultados
output = {
    'original': df_original,
    'processado': df_processado
}

print("\nSalvando resultados...")
try:
    joblib.dump(output, 'padronizado_produtos.z')
    print("✔ Arquivo 'padronizado_produtos.z' salvo com sucesso!")
    print("\nEstrutura do arquivo salvo:")
    print(f"- Original: {df_original.shape} linhas, {len(df_original.columns)} colunas")
    print(f"- Processado: {df_processado.shape} linhas, {len(df_processado.columns)} colunas")
except Exception as e:
    print(f"Erro ao salvar arquivo: {e}")

# 5. Amostra dos resultados
print("\nAmostra dos dados processados:")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
display(df_processado.head(3))

Carregando dados...
✔ Dados carregados. Total de linhas: 9,199

Processando dados...


Convertendo texto: 100%|██████████| 4/4 [00:00<00:00, 412.79it/s]


Verificando transformações...

Verificação de minúsculas:
descricao    True
unidade      True
marca        True
categoria    True
dtype: bool

Estatísticas dos preços:
       preco_venda  preco_padronizado  preco_normalizado
count    9199.00         9199.00            9199.00     
mean      317.00           -0.00               0.01     
std      1394.42            1.00               0.03     
min         0.15           -0.23               0.00     
25%        22.51           -0.21               0.00     
50%        71.42           -0.18               0.00     
75%       224.99           -0.07               0.00     
max     51283.64           36.55               1.00     

Padronização - Média: -0.00 (deveria ser ~0), Desvio: 1.00 (deveria ser ~1)
Normalização - Mínimo: 0.00, Máximo: 1.00 (deveria estar entre 0-1)

Salvando resultados...
✔ Arquivo 'padronizado_produtos.z' salvo com sucesso!

Estrutura do arquivo salvo:
- Original: (9199, 7) linhas, 7 colunas
- Processado: (9199, 9) li

,id,descricao,unidade,marca,codigo_categoria,categoria,preco_venda,preco_padronizado,preco_normalizado
1,194412,caneta ref 35633,un,bic,342,caneta,8.93,-0.220944,0.000171
3,006467,caneta ref 5968,un,bic,342,caneta,8.38,-0.221338,0.000160
4,004559,caneta ref 4092,un,bic,342,caneta,67.91,-0.178644,0.001321


In [84]:
import joblib
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Carregar os dados
dados = joblib.load('padronizado_produtos.z')
df_original = dados['original']
df_processado = dados['processado']

# Configuração de exibição
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 30)

# 1. Mostrar dados originais
print("\n" + "="*50)
print("DADOS ORIGINAIS (BRUTOS)")
print("="*50)
display(df_original.head(10))

# 2. Mostrar dados processados
print("\n" + "="*50)
print("DADOS PROCESSADOS (ATUALIZADOS)")
print("="*50)
display(df_processado.head(10))

# 3. Resultados das transformações numéricas
if 'preco_padronizado' in df_processado.columns:
    print("\n" + "="*50)
    print("RESULTADOS DA PADRONIZAÇÃO (STANDARDSCALER)")
    print("="*50)
    stats_pad = df_processado['preco_padronizado'].describe().to_frame('Valor')

    verificacao_pad = pd.Series({
        'mean': f"Média ~0? {'✔' if -0.1 < stats_pad.loc['mean', 'Valor'] < 0.1 else '✖'}",
        'std': f"Desvio ~1? {'✔' if 0.9 < stats_pad.loc['std', 'Valor'] < 1.1 else '✖'}"
    }, name='Verificação')

    display(stats_pad.round(2).join(verificacao_pad))

if 'preco_normalizado' in df_processado.columns:
    print("\n" + "="*50)
    print("RESULTADOS DA NORMALIZAÇÃO (MINMAXSCALER)")
    print("="*50)
    stats_norm = df_processado['preco_normalizado'].describe().to_frame('Valor')

    verificacao_norm = pd.Series({
        'min': f"Mínimo >=0? {'✔' if stats_norm.loc['min', 'Valor'] >= 0 else '✖'}",
        'max': f"Máximo <=1? {'✔' if stats_norm.loc['max', 'Valor'] <= 1 else '✖'}"
    }, name='Verificação')

    display(stats_norm.round(2).join(verificacao_norm))

# 4. Verificação das transformações textuais
print("\n" + "="*50)
print("VERIFICAÇÃO DE TEXTO EM MINÚSCULAS")
print("="*50)
text_cols = ['descricao', 'unidade', 'marca', 'categoria']
text_check = df_processado[text_cols].apply(lambda x: x.str.islower().all())
display(text_check.to_frame('Tudo em minúsculas?'))

# 5. Comparação lado a lado
print("\n" + "="*50)
print("COMPARAÇÃO DETALHADA (PRIMEIRAS 5 LINHAS)")
print("="*50)
comparacao = pd.concat([
    df_original.head().add_prefix('ORIG_'),
    df_processado.head().add_prefix('PROC_')
], axis=1)
display(comparacao)


DADOS ORIGINAIS (BRUTOS)


,id,descricao,unidade,marca,codigo_categoria,categoria,preco_venda
1,194412,CANETA REF 35633,UN,BIC,342,CANETA,8.93
3,006467,CANETA REF 5968,UN,BIC,342,CANETA,8.38
4,004559,CANETA REF 4092,UN,BIC,342,CANETA,67.91
5,006654,ENVELOPE REF 6152,UN,3M,9,ENVELOPE,441.54
6,005543,CANETA REF 5066,UN,BIC,342,CANETA,14.74
7,005422,CANETA REF 4947,UN,BIC,342,CANETA,26.52
8,193691,TINTA REF 34938,UN,FABER CASTELL,7,TINTA,954.28
9,161291,CANETA REF 16450,UN,BIC,342,CANETA,16.00
10,006747,LIVRO REF 6244,UN,EDIOURO,2,LIVRO,81.18
11,000887,TINTA REF 860,UN,FABER CASTELL,7,TINTA,62.00



DADOS PROCESSADOS (ATUALIZADOS)


,id,descricao,unidade,marca,codigo_categoria,categoria,preco_venda,preco_padronizado,preco_normalizado
1,194412,caneta ref 35633,un,bic,342,caneta,8.93,-0.220944,0.000171
3,006467,caneta ref 5968,un,bic,342,caneta,8.38,-0.221338,0.000160
4,004559,caneta ref 4092,un,bic,342,caneta,67.91,-0.178644,0.001321
5,006654,envelope ref 6152,un,3m,9,envelope,441.54,0.089317,0.008607
6,005543,caneta ref 5066,un,bic,342,caneta,14.74,-0.216777,0.000284
7,005422,caneta ref 4947,un,bic,342,caneta,26.52,-0.208328,0.000514
8,193691,tinta ref 34938,un,faber castell,7,tinta,954.28,0.457046,0.018605
9,161291,caneta ref 16450,un,bic,342,caneta,16.00,-0.215873,0.000309
10,006747,livro ref 6244,un,ediouro,2,livro,81.18,-0.169127,0.001580
11,000887,tinta ref 860,un,faber castell,7,tinta,62.00,-0.182883,0.001206



RESULTADOS DA PADRONIZAÇÃO (STANDARDSCALER)


...

,Valor,Verificação
count,9199.00,NaN
mean,0.01,NaN
std,0.03,NaN
min,0.00,Mínimo >=0? ✔
25%,0.00,NaN
50%,0.00,NaN
75%,0.00,NaN
max,1.00,Máximo <=1? ✔



VERIFICAÇÃO DE TEXTO EM MINÚSCULAS


,Tudo em minúsculas?
descricao,True
unidade,True
marca,True
categoria,True



COMPARAÇÃO DETALHADA (PRIMEIRAS 5 LINHAS)


,ORIG_id,ORIG_descricao,ORIG_unidade,ORIG_marca,ORIG_codigo_categoria,ORIG_categoria,ORIG_preco_venda,PROC_id,PROC_descricao,PROC_unidade,PROC_marca,PROC_codigo_categoria,PROC_categoria,PROC_preco_venda,PROC_preco_padronizado,PROC_preco_normalizado
1,194412,CANETA REF 35633,UN,BIC,342,CANETA,8.93,194412,caneta ref 35633,un,bic,342,caneta,8.93,-0.220944,0.000171
3,006467,CANETA REF 5968,UN,BIC,342,CANETA,8.38,006467,caneta ref 5968,un,bic,342,caneta,8.38,-0.221338,0.000160
4,004559,CANETA REF 4092,UN,BIC,342,CANETA,67.91,004559,caneta ref 4092,un,bic,342,caneta,67.91,-0.178644,0.001321
5,006654,ENVELOPE REF 6152,UN,3M,9,ENVELOPE,441.54,006654,envelope ref 6152,un,3m,9,envelope,441.54,0.089317,0.008607
6,005543,CANETA REF 5066,UN,BIC,342,CANETA,14.74,005543,caneta ref 5066,un,bic,342,caneta,14.74,-0.216777,0.000284
